In [1]:
import syft as sy
import torch as th

#!pip install -e . ## install local editable copy

%load_ext autoreload
%autoreload 2

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/mshang/Desktop/dev_env/openMined/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [2]:
# verify is using local editable copy
!pip show syft

Name: syft
Version: 0.2.1a1
Summary: A Library for Private, Secure Deep Learning
Home-page: https://github.com/OpenMined/PySyft
Author: Andrew Trask
Author-email: contact@openmined.org
License: Apache-2.0
Location: /Users/mshang/Desktop/dev_env/openMined/PySyft
Requires: Flask, flask-socketio, syft-proto, lz4, msgpack, numpy, tblib, torch, torchvision, websocket-client, websockets, zstd, phe
Required-by: 


In [2]:
hook = sy.TorchHook(th)
hook.local_worker.is_client_worker = False

In [31]:
@plan(sy.placeholder.FloatTensor(2,3), sy.placeholder.FloatTensor((3,5)))
def my_plan(arg_1, arg_2):
    output = arg_1 + arg_2
    return output

plan_ptr = my_plan.send(bob)

NameError: name 'plan' is not defined

In [ ]:
'''experimental code'''

In [3]:

'''MVP for init promise tensor on worker'''

# TODOS: 
# 1. fix "multiple values for shape error" when doing test.promise_tensor().FloatTensor(shape=th.Size((3, 3))

test = sy.VirtualWorker(hook=hook, id="test")
test2 = sy.VirtualWorker(hook=hook, id="test2")

r = test.promise_tensor().FloatTensor()
print('r ======>', r)
print('test ======>', test)

n obj before 0
34 promise messaging 1 <VirtualWorker id:test #objects:0> 2 None
n obj after 1
owner is None
34 promise messaging 1 <VirtualWorker id:me #objects:0> 2 torch.FloatTensor
r ======> (Wrapper)>[PromiseTensor(me:73906466585) -future-> type: FloatTensor -blocking-> plans: None plans]
test ======> <VirtualWorker id:test #objects:1>


In [27]:
# print(test._objects)


In [34]:
r.keep(th.ones((3,3)))

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [30]:

'''MVP for move extends computational graph '''
# TODOS: 
# 1. make it work a promise tensor initated on worker
# 2. change method to extend instead of move and doing hacky promise type check
# 3. original promise tensor should point to new promise tensor when calling extend. 
#    i.e. PT(1)@Obj1 => call extend to obj2 => pointerToPT1 at location obj1, PT1@obj2



promise_tensor = sy.Promise.FloatTensor(shape=th.Size((3, 3)))
floaty =  th.tensor([1,2,3,4,5])

floaty_ptr = floaty.send(test)
promise_ptr =promise_tensor.send(test)

print('*** float | promise pointers', floaty_ptr, promise_ptr)

print('*** moved float tensor pointer', floaty_ptr.move(test2))
print('*** moved/extended promise_pointr', promise_ptr.move(test2))

print('=================')
print(test._objects)
print('=================')
print(test2._objects)

owner is None
34 promise messaging 1 <VirtualWorker id:me #objects:185> 2 torch.FloatTensor
native send tensor([1, 2, 3, 4, 5]) <VirtualWorker id:test #objects:32>
native, self.owner <VirtualWorker id:me #objects:187> True
native send (Wrapper)>[PromiseTensor(me:63927114088) -future-> type: FloatTensor -blocking-> plans: None plans] <VirtualWorker id:test #objects:33>
native, self.owner <VirtualWorker id:me #objects:189> False
49 ter-promise prints shape torch.Size([3, 3])
49 ter-promise prints shape torch.Size([3, 3])
34 promise messaging 1 <VirtualWorker id:test #objects:33> 2 torch.FloatTensor
*** float | promise pointers (Wrapper)>[PointerTensor | me:49561174820 -> test:87568417193] (Wrapper)>[PointerTensor | me:60492009153 -> test:63927114088]
ptr tensor.move tensor([1, 2, 3, 4, 5]) <VirtualWorker id:test #objects:34> False
*** moved float tensor pointer (Wrapper)>[PointerTensor | me:10415619875 -> test2:49561174820]
ptr tensor.move (Wrapper)>[PromiseTensor(test:63927114088) -futu

In [16]:
'''end of experimental code'''

'end of experimental code'

In [8]:
test = sy.VirtualWorker(hook=hook, id="test")
rm = sy.VirtualWorker(hook=hook, id="rm")

a = sy.Promise.FloatTensor(shape=th.Size((3, 3)))
b = sy.Promise.FloatTensor(shape=th.Size((3, 3)))

print(a)

ref = a.send(test) #base def send, inter => native def send
bref = b.extend(rm)

owner is None
generic.tensor True
generic.tensor is_promise True
34 promise messaging 1 <VirtualWorker id:me #objects:11> 2 torch.FloatTensor
owner is None
generic.tensor True
generic.tensor is_promise True
34 promise messaging 1 <VirtualWorker id:me #objects:12> 2 torch.FloatTensor
(Wrapper)>[PromiseTensor(me:74489990928) -future-> type: FloatTensor -blocking-> plans: None plans]
native send
49 ter-promise prints shape torch.Size([3, 3])
generic.tensor False
generic.tensor is_promise False
49 ter-promise prints shape torch.Size([3, 3])
generic.tensor True
generic.tensor is_promise True
34 promise messaging 1 <VirtualWorker id:test #objects:3> 2 torch.FloatTensor


AttributeError: 'Tensor' object has no attribute 'extend'

In [29]:
print(ref)
print(test._objects)

(Wrapper)>[PointerTensor | me:79175181073 -> test:17371698329]
{75011036619: (Wrapper)>[PromiseTensor(test:75011036619) -future-> FloatTensor -blocking-> 0 plans], 6485164871: (Wrapper)>[PromiseTensor(test:6485164871) -future-> FloatTensor -blocking-> 0 plans], 35378247926: (Wrapper)>[PromiseTensor(test:35378247926) -future-> FloatTensor -blocking-> 0 plans], 50030411072: (Wrapper)>[PromiseTensor(test:50030411072) -future-> FloatTensor -blocking-> 0 plans], 17371698329: (Wrapper)>[PromiseTensor(test:17371698329) -future-> FloatTensor -blocking-> 0 plans]}


In [ ]:
x = a.send(bob)

In [8]:
president = sy.VirtualWorker(hook=hook, id="president")
vice_president = sy.VirtualWorker(hook=hook, id="vice_president")
secretary = sy.VirtualWorker(hook=hook, id="secretary")

In [9]:
x = president.Promise.FloatTensor((3,5))
y = vice_president.Promise.FloatTensor((3,5))

AttributeError: 'VirtualWorker' object has no attribute 'Promise'

In [5]:
x2 = x.move(secretary)
y2 = y.move(secretary)

In [6]:
z = x2 + y2

In [7]:
pres_output = z.move(president)
vice_pres_output = z.move(vice_president)

In [ ]:
pres_output.declare_output(0)
vice_pres_output.declare_output(0)

In [8]:
x.keep(th.ones((3,5)), keep_id="jaw23g23")
y.keep(th.ones((3,5)), keep_id="2352351")

# still blocking for both

x.keep(th.ones((3,5)), keep_id="2352351") # this execution would finish
y.keep(th.ones((3,5)), keep_id="jaw23g23") # this execution would finish

[PointerTensor | me:44509726196 -> secretary:74544206441]

### Wishlist:

1. initialization of PromiseTensor objects on remote workers
2. I want to be able to call .move() on a remote tensor, and instead of it moving the PromiseTensor, I want it to actual extend the PromiseTensor graph across workers. 
3. I want to be able to call .keep() multiple times.
4. I want to be able to call .declare_output() so that sy.Protocol() can know which Promises are each player's output.
5. Eliminate need to set "hook.local_worker.is_client_worker = False"

In [12]:
third_party_addition = sy.Protocol(president, vice_president, secretary)

In [13]:
trump = sy.VirtualWorker(hook=hook, id="Donald Trump")
pence = sy.VirtualWorker(hook=hook, id="Mike Pence")
mike = sy.VirtualWorker(hook=hook, id="Mike Pompeo")

In [15]:
{"president": (x_ptr), "vice_president": (y_ptr,)},
{"president": (pre_output), "vice_president": (vp_output)} = third_party_addition.distribute(president=trump, 
                                                                                   vice_president=pence, 
                                                                                   secretary=mike)

x_ptr.keep(th.tensor([2,3,4,5,6], keep_id="123214").send(trump))
y_ptr.keep(th.tensor([1,1,1,1,1], keep_id="123214").send(pence))

x_ptr.keep(pre_output.value("123214"), keep_id="a23523")
y_ptr.keep(vp_output.value("123214"), keep_id="a23523")

In [ ]:
pointers_to_input_promises = pygrid_node.host(third_party_addition)